### Скрипт для парсинга и анализа комментариев в Telegram-канале с сохранением в SQLite базу данных.

In [1]:
import os
from dotenv import load_dotenv
import asyncio
import logging
import aiosqlite
from telethon import TelegramClient
from telethon.tl.functions.messages import GetRepliesRequest
from telethon.errors import FloodWaitError
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

api_id = int(os.getenv('API_ID'))
api_hash = os.getenv('API_HASH')

async def main():
    async with TelegramClient('session_name1', api_id, api_hash) as client:
        print("✅ Успешная авторизация!")

await main()

✅ Успешная авторизация!


In [2]:
channel_username = 'moscowmap'
DB_PATH = "telegram_data3.db"  

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler("parser.log"), logging.StreamHandler()]
)

client = TelegramClient('session_name1', api_id, 
                        api_hash)

async def init_db():
    async with aiosqlite.connect(DB_PATH) as db:
        await db.execute("""
            CREATE TABLE IF NOT EXISTS comments (
                id INTEGER PRIMARY KEY,
                post_id INTEGER,
                user_id INTEGER,
                text TEXT,
                date TEXT,
                spam BOOLEAN DEFAULT 0
            )
        """)
        await db.commit()
        logging.info("База данных инициализирована.")

async def fetch_and_track_comments(post_id: int, check_delays: list[int], channel_entity):
    logging.info(f"Отслеживаем комментарии к посту {post_id}")
    seen_comments = {}

    async def check_comments():
        nonlocal seen_comments
        try:
            replies = await client(GetRepliesRequest(
                peer=channel_entity,
                msg_id=post_id,
                offset_id=0,
                offset_date=None,
                add_offset=0,
                limit=200,
                max_id=0,
                min_id=0,
                hash=0
            ))

            current_ids = set()

            async with aiosqlite.connect(DB_PATH) as db:
                inserted = 0
                for msg in replies.messages:
                    current_ids.add(msg.id)
                    seen_comments[msg.id] = msg
                    await db.execute("""
                        INSERT OR IGNORE INTO comments (id, post_id, user_id, text, date)
                        VALUES (?, ?, ?, ?, ?)
                    """, (
                        msg.id,
                        post_id,
                        getattr(msg.from_id, 'user_id', None),
                        msg.message,
                        msg.date.strftime('%Y-%m-%d %H:%M:%S')
                    ))
                    inserted += 1
                await db.commit()
                logging.info(f"Сохранено {inserted} комментариев для поста {post_id}")

                for old_id in list(seen_comments.keys()):
                    if old_id not in current_ids:
                        await db.execute("UPDATE comments SET spam = 1 WHERE id = ?", (old_id,))
                        logging.info(f"⚠️ Комментарий {old_id} помечен как SPAM")
                        del seen_comments[old_id]
                await db.commit()

        except FloodWaitError as e:
            logging.warning(f"Flood wait: ждём {e.seconds} секунд...")
            await asyncio.sleep(e.seconds)
        except Exception as e:
            logging.error(f"🚨 Ошибка при получении комментариев: {e}")

    for delay in check_delays:
        logging.info(f"Ждём {delay} сек перед следующей проверкой поста {post_id}")
        await asyncio.sleep(delay)
        await check_comments()

    logging.info(f"✅ Отслеживание поста {post_id} завершено.")

async def main():
    await init_db()
    await client.start()
    logging.info("Telegram клиент запущен.")
    channel_entity = await client.get_entity(channel_username)

    processed_posts = set()

    while True:
        async for message in client.iter_messages(channel_entity, limit=2):
            if message.replies and message.replies.replies > 0 and message.id not in processed_posts:
                logging.info(f"Пост с комментариями найден: {message.id}")
                processed_posts.add(message.id)  
                asyncio.create_task(fetch_and_track_comments(message.id, [30, 60, 300, 600], channel_entity))

        await asyncio.sleep(60)

if __name__ == "__main__":
    try:
        asyncio.get_event_loop().run_until_complete(main())
    except KeyboardInterrupt:
        logging.info("Прервано пользователем (Ctrl+C)")
    except Exception as e:
        logging.critical(f"Критическая ошибка: {e}")

2025-05-07 20:49:30,305 | INFO | База данных инициализирована.
2025-05-07 20:49:30,309 | INFO | Connecting to 149.154.167.51:443/TcpFull...


2025-05-07 20:49:30,360 | INFO | Connection to 149.154.167.51:443/TcpFull complete!
2025-05-07 20:49:30,682 | INFO | Telegram клиент запущен.
2025-05-07 20:49:30,826 | INFO | Пост с комментариями найден: 69053
2025-05-07 20:49:30,827 | INFO | Пост с комментариями найден: 69051
2025-05-07 20:49:30,829 | INFO | Отслеживаем комментарии к посту 69053
2025-05-07 20:49:30,832 | INFO | Ждём 30 сек перед следующей проверкой поста 69053
2025-05-07 20:49:30,833 | INFO | Отслеживаем комментарии к посту 69051
2025-05-07 20:49:30,834 | INFO | Ждём 30 сек перед следующей проверкой поста 69051
2025-05-07 20:50:01,214 | INFO | Сохранено 100 комментариев для поста 69053
2025-05-07 20:50:01,218 | INFO | Ждём 60 сек перед следующей проверкой поста 69053
2025-05-07 20:50:01,256 | INFO | Сохранено 100 комментариев для поста 69051
2025-05-07 20:50:01,259 | INFO | Ждём 60 сек перед следующей проверкой поста 69051
2025-05-07 20:51:01,665 | INFO | Сохранено 99 комментариев для поста 69053
2025-05-07 20:51:01,6